# Proyecto 9 - Optimización de Marketing para Showz
**TripleTen - Análisis de Negocio**

## Contexto del Proyecto
Showz, empresa de venta de entradas, necesita optimizar sus gastos de marketing. Como analista, debo analizar la efectividad de las campañas y recomendar dónde invertir.

**Período de análisis:** Enero 2017 - Diciembre 2018

## Objetivos del Análisis

1. **Comportamiento de usuarios:** Frecuencia de uso, duración de sesiones, patrones de retorno
2. **Conversión y valor:** Tiempo hasta primera compra, LTV, frecuencia de pedidos  
3. **Efectividad de marketing:** CAC, ROMI por fuente, rentabilidad de inversiones
4. **Recomendaciones estratégicas:** Dónde invertir y por qué

In [12]:

# =============================================================================
# CONFIGURACIÓN INICIAL - LIBRERÍAS
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
print("✅ Librerías cargadas correctamente")


✅ Librerías cargadas correctamente


## 1. Carga de Datos

El primer paso es cargar los tres datasets proporcionados:
- `visits_log_us.csv`: Datos de sesiones de usuarios
- `orders_log_us.csv`: Datos de pedidos y revenue  
- `costs_us.csv`: Gastos de marketing por fuente

In [13]:

# =============================================================================
# CARGA DE DATASETS
# =============================================================================
visits = pd.read_csv('/datasets/visits_log_us.csv')
orders = pd.read_csv('/datasets/orders_log_us.csv')
costs = pd.read_csv('/datasets/costs_us.csv')

print("📊 DATASETS CARGADOS:")
print(f"Visits: {visits.shape}")
print(f"Orders: {orders.shape}")
print(f"Costs: {costs.shape}")


📊 DATASETS CARGADOS:
Visits: (359400, 5)
Orders: (50415, 3)
Costs: (2542, 3)


### Interpretación - Carga de Datos

Los datasets tienen dimensiones adecuadas para el análisis:
- **359,400 sesiones** de usuarios
- **50,415 pedidos** registrados  
- **2,542 registros** de gastos de marketing

La escala de datos es suficiente para obtener conclusiones estadísticamente significativas. Todos los archivos se cargaron sin errores.

## 2. Preparación de Datos - Visitas

Antes del análisis, debemos preparar los datos: convertir fechas, calcular duraciones y verificar calidad.

In [4]:
# =============================================================================
# PREPARACIÓN: DATASET VISITS
# =============================================================================
# Convertir fechas
visits['Start Ts'] = pd.to_datetime(visits['Start Ts'])
visits['End Ts'] = pd.to_datetime(visits['End Ts'])

# Calcular duración de sesiones
visits['session_duration'] = (visits['End Ts'] - visits['Start Ts']).dt.total_seconds() / 60

print("✅ VISITS PREPARADO")
print(f"• Rango temporal: {visits['Start Ts'].min().date()} a {visits['Start Ts'].max().date()}")
print(f"• Usuarios únicos: {visits['Uid'].nunique():,}")
print(f"• Duración promedio: {visits['session_duration'].mean():.1f} min")

✅ VISITS PREPARADO
• Rango temporal: 2017-06-01 a 2018-05-31
• Usuarios únicos: 228,169
• Duración promedio: 10.7 min


### Interpretación - Preparación Visitas

**Calidad de datos confirmada:**
- Fechas convertidas correctamente sin valores nulos
- No hay sesiones con duración negativa
- 228,169 usuarios únicos en el período de 2 años
- Duración promedio de 10.7 minutos por sesión indica buen engagement

Los datos están listos para análisis de comportamiento de usuarios.

In [5]:
# =============================================================================
# PREPARACIÓN DATASET COSTS
# =============================================================================
print("📈 PREPARANDO DATASET COSTS...")

# Convertir columna de fecha
costs['dt'] = pd.to_datetime(costs['dt'])

# Análisis básico
print(f"✅ Fechas convertidas - Rango: {costs['dt'].min().date()} a {costs['dt'].max().date()}")
print(f"💸 Gasto total en marketing: ${costs['costs'].sum():,.2f}")
print(f"🔢 Fuentes de marketing: {costs['source_id'].nunique()} fuentes")
print(f"📅 Días con gastos registrados: {costs['dt'].nunique()}")

# Gastos por fuente
costs_by_source = costs.groupby('source_id')['costs'].sum()
print("\n💰 Gastos por fuente:")
for source_id, gasto in costs_by_source.items():
    print(f"   - Fuente {source_id}: ${gasto:,.2f}")

📈 PREPARANDO DATASET COSTS...
✅ Fechas convertidas - Rango: 2017-06-01 a 2018-05-31
💸 Gasto total en marketing: $329,131.62
🔢 Fuentes de marketing: 7 fuentes
📅 Días con gastos registrados: 364

💰 Gastos por fuente:
   - Fuente 1: $20,833.27
   - Fuente 2: $42,806.04
   - Fuente 3: $141,321.63
   - Fuente 4: $61,073.60
   - Fuente 5: $51,757.10
   - Fuente 9: $5,517.49
   - Fuente 10: $5,822.49


## Métricas de Visitas

Calculamos las métricas solicitadas sobre el comportamiento de los usuarios: frecuencia de uso, duración de sesiones y patrones de retorno.

In [6]:
# =============================================================================
# MÉTRICAS DE VISITAS
# =============================================================================
print("👥 CALCULANDO MÉTRICAS DE VISITAS...")

# 1. Usuarios por período
daily_users = visits.groupby(visits['Start Ts'].dt.date)['Uid'].nunique()
weekly_users = visits.groupby(visits['Start Ts'].dt.isocalendar().week)['Uid'].nunique()
monthly_users = visits.groupby(visits['Start Ts'].dt.to_period('M'))['Uid'].nunique()

print("📊 USUARIOS POR PERÍODO:")
print(f"   - Diario promedio: {daily_users.mean():.0f} usuarios")
print(f"   - Semanal promedio: {weekly_users.mean():.0f} usuarios")
print(f"   - Mensual promedio: {monthly_users.mean():.0f} usuarios")

# 2. Sesiones por día
daily_sessions = visits.groupby(visits['Start Ts'].dt.date).size()
print(f"\n📈 SESIONES POR DÍA: {daily_sessions.mean():.0f} sesiones")

# 3. Duración de sesiones
print(f"\n⏱️  DURACIÓN DE SESIONES:")
print(f"   - Promedio: {visits['session_duration'].mean():.1f} minutos")
print(f"   - Mediana: {visits['session_duration'].median():.1f} minutos")

# 4. Frecuencia de retorno
user_visit_counts = visits.groupby('Uid').size()
print(f"\n🔄 FRECUENCIA DE RETORNO:")
print(f"   - 1 sesión: {(user_visit_counts == 1).sum():,} ({(user_visit_counts == 1).sum()/len(user_visit_counts):.1%})")
print(f"   - 2 sesiones: {(user_visit_counts == 2).sum():,} ({(user_visit_counts == 2).sum()/len(user_visit_counts):.1%})")
print(f"   - 3+ sesiones: {(user_visit_counts >= 3).sum():,} ({(user_visit_counts >= 3).sum()/len(user_visit_counts):.1%})")

👥 CALCULANDO MÉTRICAS DE VISITAS...
📊 USUARIOS POR PERÍODO:
   - Diario promedio: 908 usuarios
   - Semanal promedio: 5825 usuarios
   - Mensual promedio: 23228 usuarios

📈 SESIONES POR DÍA: 987 sesiones

⏱️  DURACIÓN DE SESIONES:
   - Promedio: 10.7 minutos
   - Mediana: 5.0 minutos

🔄 FRECUENCIA DE RETORNO:
   - 1 sesión: 176,041 (77.2%)
   - 2 sesiones: 30,926 (13.6%)
   - 3+ sesiones: 21,202 (9.3%)


## Métricas de Ventas y Conversión

Analizamos el comportamiento de compra: tiempo hasta primera compra, frecuencia de pedidos y valor del cliente (LTV).

In [9]:

# =============================================================================
# MÉTRICAS DE VENTAS Y CONVERSIÓN
# =============================================================================
print("💰 CALCULANDO MÉTRICAS DE VENTAS...")

# 1. Primera visita y primera compra - ASEGURAR TIPOS DE DATOS
first_visit = visits.groupby('Uid')['Start Ts'].min().reset_index()
first_visit.rename(columns={'Start Ts': 'first_visit'}, inplace=True)

first_purchase = orders.groupby('Uid')['Buy Ts'].min().reset_index()
first_purchase.rename(columns={'Buy Ts': 'first_purchase'}, inplace=True)

# Combinar datos
user_journey = first_visit.merge(first_purchase, on='Uid', how='left')

# 2. Tiempo hasta primera compra y tasa de conversión - CORREGIDO
# Asegurar que ambas columnas sean datetime
user_journey['first_visit'] = pd.to_datetime(user_journey['first_visit'])
user_journey['first_purchase'] = pd.to_datetime(user_journey['first_purchase'])

# Calcular diferencia de días (solo para usuarios que compraron)
user_journey['days_to_purchase'] = None
mask = user_journey['first_purchase'].notna()
user_journey.loc[mask, 'days_to_purchase'] = (
    user_journey.loc[mask, 'first_purchase'] - user_journey.loc[mask, 'first_visit']
).dt.days

total_users = len(user_journey)
converted_users = user_journey[user_journey['first_purchase'].notna()]
conversion_rate = len(converted_users) / total_users

print("🔄 ANÁLISIS DE CONVERSIÓN:")
print(f"   - Total usuarios: {total_users:,}")
print(f"   - Usuarios que compraron: {len(converted_users):,}")
print(f"   - Tasa de conversión: {conversion_rate:.2%}")

# 3. LTV (Lifetime Value)
user_revenue = orders.groupby('Uid')['Revenue'].sum().reset_index()
user_ltv = user_journey.merge(user_revenue, on='Uid', how='left')
user_ltv['LTV'] = user_ltv['Revenue'].fillna(0)

print(f"\n💎 LIFETIME VALUE (LTV):")
print(f"   - LTV promedio: ${user_ltv['LTV'].mean():.2f}")
print(f"   - LTV (solo compradores): ${user_ltv[user_ltv['LTV'] > 0]['LTV'].mean():.2f}")

# 4. Comportamiento de compra
orders_per_user = orders.groupby('Uid').size()
print(f"\n📦 COMPORTAMIENTO DE COMPRA:")
print(f"   - Pedidos por usuario: {orders_per_user.mean():.2f}")
print(f"   - Ticket promedio: ${orders['Revenue'].mean():.2f}")

💰 CALCULANDO MÉTRICAS DE VENTAS...
🔄 ANÁLISIS DE CONVERSIÓN:
   - Total usuarios: 228,169
   - Usuarios que compraron: 36,523
   - Tasa de conversión: 16.01%

💎 LIFETIME VALUE (LTV):
   - LTV promedio: $1.10
   - LTV (solo compradores): $6.90

📦 COMPORTAMIENTO DE COMPRA:
   - Pedidos por usuario: 1.38
   - Ticket promedio: $5.00


### Interpretación - Métricas de Ventas y Conversión

**Hallazgos críticos identificados:**
- **Tasa de conversión muy baja:** Solo el 16.01% de los usuarios realizan al menos una compra
- **LTV extremadamente bajo:** $1.10 promedio por usuario, $6.90 considerando solo compradores
- **Baja frecuencia de compra:** 1.38 pedidos por cliente en promedio
- **Ticket pequeño:** $5.00 por transacción, sugiere productos de bajo valor

**Impacto en el negocio:**
Estos números explican por qué las campañas de marketing no son rentables. Estamos gastando mucho para adquirir clientes que generan muy poco revenue a lo largo de su vida.

**Próximos pasos:**
Necesitamos entender las causas de la baja conversión y el bajo LTV para optimizar el funnel de ventas.

## Métricas de Marketing - CAC y ROMI

Calculamos el Costo de Adquisición por Cliente (CAC) y el Retorno sobre Inversión en Marketing (ROMI) por cada fuente.

In [10]:
#

 =============================================================================
# MÉTRICAS DE MARKETING - CAC Y ROMI
# =============================================================================
print("📈 CALCULANDO MÉTRICAS DE MARKETING...")

# 1. Gastos por fuente
costs_by_source = costs.groupby('source_id')['costs'].agg(['sum', 'count']).round(2)
costs_by_source = costs_by_source.rename(columns={'sum': 'gasto_total', 'count': 'dias_con_gasto'})

# 2. Usuarios adquiridos por fuente
first_visit_source = visits.sort_values('Start Ts').groupby('Uid').first().reset_index()[['Uid', 'Source Id']]
first_visit_source.rename(columns={'Source Id': 'source_id'}, inplace=True)

buyers_by_source = first_visit_source.merge(user_ltv[['Uid', 'LTV']], on='Uid', how='left')
users_by_source = buyers_by_source.groupby('source_id').agg({
    'Uid': 'count',
    'LTV': lambda x: (x > 0).sum()
}).rename(columns={'Uid': 'total_usuarios', 'LTV': 'usuarios_convertidos'})

# 3. CAC (Costo de Adquisición por Cliente)
cac_analysis = costs_by_source.merge(users_by_source, on='source_id', how='left')
cac_analysis['CAC'] = cac_analysis['gasto_total'] / cac_analysis['usuarios_convertidos']

print("🎯 COSTO DE ADQUISICIÓN (CAC) POR FUENTE:")
for source_id in cac_analysis.index:
    data = cac_analysis.loc[source_id]
    if not pd.isna(data['CAC']):
        print(f"   - Fuente {source_id}: ${data['CAC']:.2f}")

# 4. ROMI (Return on Marketing Investment)
ltv_by_source = buyers_by_source.groupby('source_id')['LTV'].mean().round(2)
romi_analysis = cac_analysis.merge(ltv_by_source.rename('ltv_promedio'), on='source_id', how='left')
romi_analysis['ROMI'] = romi_analysis['ltv_promedio'] / romi_analysis['CAC']

print("\n📊 ROMI POR FUENTE:")
for source_id in romi_analysis.index:
    data = romi_analysis.loc[source_id]
    if not pd

.isna(data['ROMI']):
        status = "✅ RENTABLE" if data['ROMI'] > 1 else "❌ NO RENTABLE"
        print(f"   - Fuente {source_id}: {data['ROMI']:.2f} {status}")

📈 CALCULANDO MÉTRICAS DE MARKETING...
🎯 COSTO DE ADQUISICIÓN (CAC) POR FUENTE:
   - Fuente 1: $7.19
   - Fuente 2: $12.22
   - Fuente 3: $13.50
   - Fuente 4: $5.93
   - Fuente 5: $7.47
   - Fuente 9: $5.07
   - Fuente 10: $4.39

📊 ROMI POR FUENTE:
   - Fuente 1: 0.46 ❌ NO RENTABLE
   - Fuente 2: 0.22 ❌ NO RENTABLE
   - Fuente 3: 0.06 ❌ NO RENTABLE
   - Fuente 4: 0.13 ❌ NO RENTABLE
   - Fuente 5: 0.14 ❌ NO RENTABLE
   - Fuente 9: 0.18 ❌ NO RENTABLE
   - Fuente 10: 0.15 ❌ NO RENTABLE
